In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import random
import matplotlib.pyplot as plt
import cv2
import json

In [2]:
images_path = "OpenKidney/images/"
masks_path = "OpenKidney/masks/"

labels_file = "OpenKidney/reviewed_labels_2.csv"

labels = pd.read_csv(labels_file)
print(labels.shape)
labels.head()

(1689, 7)


,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1_IM-0001-0059_anon.png,140645,"{""Quality"":""Fair"",""View"":""Transverse"",""Comment...",4,0,"{""name"":""polygon"",""all_points_x"":[204,210,216,...","{""Anatomy"":""Capsule""}"
1,1_IM-0001-0059_anon.png,140645,"{""Quality"":""Fair"",""View"":""Transverse"",""Comment...",4,1,"{""name"":""polygon"",""all_points_x"":[219,219,214,...","{""Anatomy"":""Central Echo Complex""}"
2,1_IM-0001-0059_anon.png,140645,"{""Quality"":""Fair"",""View"":""Transverse"",""Comment...",4,2,"{""name"":""polygon"",""all_points_x"":[226,215,208,...","{""Anatomy"":""Medulla""}"
3,1_IM-0001-0059_anon.png,140645,"{""Quality"":""Fair"",""View"":""Transverse"",""Comment...",4,3,"{""name"":""polygon"",""all_points_x"":[313,309,311,...","{""Anatomy"":""Cortex""}"
4,100_IM-0246-0030_anon.png,178160,"{""Quality"":""Fair"",""View"":""Transverse"",""Comment...",2,0,"{""name"":""polygon"",""all_points_x"":[466,436,411,...","{""Anatomy"":""Capsule""}"


In [3]:
# get unique image names
image_names = labels["filename"].unique()

print(len(image_names))

for index in range(len(image_names)):

    image_name = image_names[index]
    # print(image_name)

    # get all labels for this image
    image_labels = labels[labels["filename"] == image_name]
    # print(image_labels.shape)

    image_labels.head()

    # get image path
    image_path = os.path.join(images_path, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # plt.imshow(image, cmap="gray")
    # plt.axis("off")
    # plt.show()

    mask = np.zeros_like(image)
    save_flag = True

    # plot mask from labels
    for i, (id, row) in enumerate(image_labels.iterrows()):
        dict = json.loads(row["region_shape_attributes"])

        if not dict:
            save_flag = False
            break

        x_points = dict["all_points_x"]
        y_points = dict["all_points_y"]

        points = np.array([x_points, y_points]).T

        # print(i)

            # print(points)
        # cv2.fillPoly(mask, pts=[points], color=int(np.clip(50 * (i + 1), 0, 255)))\
        cv2.fillPoly(mask, pts=[points], color=255)

    if not save_flag:
        continue
    mask_name = image_name
    mask_path = os.path.join(masks_path, mask_name)
    cv2.imwrite(mask_path, mask)

    # plt.imshow(mask, cmap="gray")
    # plt.axis("off")
    # plt.show()


534


In [4]:
dataset_path = "OpenKidney"

train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

# make mask and image folders
train_mask_path = os.path.join(train_path, "mask")
train_image_path = os.path.join(train_path, "image")
test_mask_path = os.path.join(test_path, "mask")
test_image_path = os.path.join(test_path, "image")

if not os.path.exists(train_mask_path):
    os.makedirs(train_mask_path)
if not os.path.exists(train_image_path):
    os.makedirs(train_image_path)
if not os.path.exists(test_mask_path):
    os.makedirs(test_mask_path)
if not os.path.exists(test_image_path):
    os.makedirs(test_image_path)

In [5]:
from sklearn.model_selection import train_test_split

# get images list
masks = os.listdir(os.path.join(dataset_path, "masks"))
print("masks num: ", len(masks))

# split images
train_masks, test_masks = train_test_split(masks, test_size=0.2, random_state=42)

masks num:  487


In [ ]:
# copy images and masks to train and test folders

for image in train_masks:
    shutil.copy(os.path.join(dataset_path, "images", image), train_image_path)
    shutil.copy(os.path.join(dataset_path, "masks", image), train_mask_path)

for image in test_masks:
    shutil.copy(os.path.join(dataset_path, "images", image), test_image_path)
    shutil.copy(os.path.join(dataset_path, "masks", image), test_mask_path)